# Hierarchical Navigable Small Worlds - HNSW
* [Hierarchical Navigable Small Worlds - HNSW](https://zilliz.com/blog/hierarchical-navigable-small-worlds-HNSW)
* [Vector Search](https://github.com/fzliu/vector-search/)
* [Reference code](https://github.com/fzliu/vector-search/blob/main/indexes/hnsw.py)
* [Reference notebook](https://github.com/fzliu/vector-search/blob/main/tutorials/2023-03-02_data_science_dojo_00_ann_algorithms.ipynb)

In [1]:
import numpy as np

dataset = np.random.normal(size=(1000, 128))

layers = 5
index = [[] for _ in range(layers)]

In [2]:
def _search_layer(graph, entry, query, ef=1):
    best = (np.linalg.norm(graph[entry][0] - query), entry)
    nearest_neighbors = [best]
    visited = set(best)
    candidates = [best]
    heapify(candidates)

    # find top-k nearest neighbors
    while candidates:
        candidate = heappop(candidates)
        if nearest_neighbors[-1][0] < candidate[0]:
            break

        # loop through all nearest neighbors to the candidate vector
        for neighbor_entry in graph[candidate[1]][1]:
            distance = np.linalg.norm(graph[neighbor_entry][0] - query)
            if (distance, neighbor_entry) not in visited:
                visited.add((distance, neighbor_entry))

                # push only "better" vectors into candidate heap
                if distance < nearest_neighbors[-1][0] or len(nearest_neighbors) < ef:
                    heappush(candidates, (distance, neighbor_entry))
                    insort(nearest_neighbors, (distance, neighbor_entry))
                    if len(nearest_neighbors) > ef:
                        nearest_neighbors.pop()

    return nearest_neighbors

In [3]:
def search(index, query, ef=1):
    if not index[0]:
        return []

    best_v = 0 # set the initial best vertex to the entry point
    for graph in index:
        best_d, best_v = _search_layer(graph, best_v, query, ef=1)[0]
        if graph[best_v][2]:
            best_v = graph[best_v][2]
        else:
            return _search_layer(graph, best_v, query, ef=ef)

In [4]:
def _get_insert_layer(layers, multi_factor):
    # multi_factor is used to normalize the distribution
    which_layer = -int(np.log(np.random.random()) * multi_factor)
    return min(which_layer, layers)

In [8]:
def insert(index, vector, efc=10):
    # if the index is empty, insert the vector into all layers and return
    if not index[0]:
        i = None
        for graph in index[::-1]:
            graph.append((vector, [], i))
            i = 0
        return

    layer = _get_insert_layer(1/np.log(layers))
    
    start_v = 0
    for n, graph in enumerate(index):
        # perform insertion for layers [layer, layers) only
        if n < layer:
            _, start_v = _search_layer(graph, start_v, vec, ef=1)[0]
        else:
            node = (vector, [], len(_index[n+1]) if n < layers-1 else None)
            nearest_neighbors = _search_layer(graph, start_v, vector, ef=efc)
            for nearest_neighbor in nearest_neighbors:
                node[1].append(nearest_neighbor[1]) # outbound connections to NNs
                graph[nearest_neighbor[1]][1].append(len(graph)) # inbound connections to node
            graph.append(node)
        # set the starting vertex to the nearest neighbor in the next layer
        start_v = graph[start_v][2]
    

In [9]:
insert(index, dataset)

In [20]:
print(len(index))

5


In [35]:
search(np.random.randn(256))

TypeError: search() missing 1 required positional argument: 'query'